<a href="https://colab.research.google.com/github/tjturnage/GIS/blob/main/Central_IA_NWA_workshop_warning_polygons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install geopandas &> /dev/null
!apt install imagemagick &> /dev/null
import os
from datetime import datetime, timedelta
import pytz
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import box
from shapely.ops import unary_union
import matplotlib.pyplot as plt

<h2>Refer to <a href="https://github.com/akrherz/nwa/blob/main/README.md" target="_blank">this page</a> to set your variables.</h2>
<h2>For example, for 2025 you will set the following:</h2>


```
shapefile_path = "https://github.com/akrherz/nwa/raw/main/output/ciowa_nwa_2025.zip"
start_time = "3/27/2025_1900"
end_time = "3/27/2025_2030"
```



In [20]:
# Load the shapefile containing polygons
shapefile_path = "https://github.com/akrherz/nwa/raw/main/output/ciowa_nwa_2025.zip"
start_time = "3/27/2025_1900"
end_time = "3/27/2025_2030"

In [ ]:
# Determine earliest and latest issue times and round down to nearest 5 minutes
start_dt = datetime.strptime(start_time, "%m/%d/%Y_%H%M")
start_dt = start_dt - timedelta(minutes=start_dt.minute % 5) # Use start_dt here to get the minute value
shifted_start_dt = start_dt + timedelta(minutes=30)
end_dt = datetime.strptime(end_time, "%m/%d/%Y_%H%M")
end_dt = end_dt - timedelta(minutes=end_dt.minute % 5) # Use end_dt here to get the minute value
shifted_end_dt = end_dt + timedelta(minutes=30)

start_tstr = start_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
end_tstr = end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
shifted_start = shifted_start_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
shifted_end = shifted_end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# Generate datetime range
start_times = pd.date_range(
    start=start_tstr,
    end=end_tstr,
    freq="5min")

end_times = pd.date_range(
    start=shifted_start,
    end=shifted_end,
    freq="5min")

# Convert to the desired ISO format
formatted_start_times = start_times.strftime("%Y-%m-%dT%H:%M:%SZ")
formatted_end_times = end_times.strftime("%Y-%m-%dT%H:%M:%SZ")


def trim_gdf(gdf, start_time, end_time):
    gdf_gt = gdf[gdf['utc_issue'] > start_time]
    gdf_lt = gdf_gt[gdf_gt['utc_issue'] < end_time]
    return gdf_lt

def make_grid(gdf, start_time, end_time):
    gdf = trim_gdf(gdf, start_time, end_time)

    # Check if the GeoDataFrame is empty after trimming
    if gdf.empty:
        print(f"Warning: GeoDataFrame is empty for start_time={start_time}, end_time={end_time}. Skipping grid creation.")
        return None  # or raise an exception if desired

    if gdf.crs is None:
        # Replace 'EPSG:YOUR_EPSG_CODE' with the actual EPSG code of the shapefile's CRS
        # If you don't know the EPSG code, you can try to find it based on the shapefile's metadata or documentation
        gdf = gdf.set_crs("EPSG:4326")

    # Ensure it's in a projected CRS for accurate distance measurement
    if gdf.crs is None or not gdf.crs.is_projected:
        gdf = gdf.to_crs("EPSG:3857")  # Web Mercator projection

    # Define grid size (2 km x 2 km)
    grid_size = 2000  # meters

    # Get the bounding box of the data
    minx, miny, maxx, maxy = gdf.total_bounds

    # Generate grid cells
    x_coords = np.arange(minx, maxx, grid_size)
    y_coords = np.arange(miny, maxy, grid_size)

    grid_cells = []
    for x in x_coords:
        for y in y_coords:
            grid_cells.append(box(x, y, x + grid_size, y + grid_size))

    # Create a GeoDataFrame for the grid
    grid_gdf = gpd.GeoDataFrame({"geometry": grid_cells}, crs=gdf.crs)

    # Count overlapping polygons in each grid cell and assign to the grid_gdf
    grid_gdf["polygon_count"] = grid_gdf.geometry.apply(lambda cell: sum(gdf.intersects(cell)))
    return grid_gdf # Return the modified grid_gdf with the 'polygon_count' column

def plot_grid(grid_gdf,start_time, end_time) -> None:
    # Plot heatmap
    temp_time = datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%SZ")
    clean_time = temp_time.strftime("%Y%m%d %H:%MZ")
    fig, ax = plt.subplots(figsize=(10, 10))


    # Overlay state boundaries
    states.boundary.plot(ax=ax, edgecolor="black", linewidth=1)

    # Overlay county boundaries
    counties.boundary.plot(ax=ax, edgecolor="gray", linewidth=0.5)

    grid_gdf.plot(column="polygon_count", cmap="hot_r", linewidth=0, vmin=0, vmax=60, edgecolor=None, legend=True, ax=ax)

    # Customize plot
    ax.set_title(f"2 km² Polygon Heatmap: 30 mins starting {clean_time}", fontsize=14)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

    # Zoom into Iowa (centered)
    # Get Iowa's centroid
    #iowa = states[states['NAME'] == 'Iowa']
    #iowa_centroid = iowa.geometry.centroid.iloc[0]
    #print(iowa_centroid)
    centroid_x, centroid_y = -10408613.911455575, 5174547.7231889395
    # Define bounding box based on Iowa's centroid (adjust buffer as needed)
    buffer = 400000  # 400 km buffer around the centroid
    buffer_y = 350000
    bounds = [
        centroid_x - buffer,
        centroid_y - buffer_y,
        centroid_x + buffer,
        centroid_y + buffer_y,
    ]

    ax.set_xlim(bounds[0], bounds[2])
    ax.set_ylim(bounds[1], bounds[3])

    plt.savefig(f"heatmap_{clean_time}.png", bbox_inches='tight', dpi=200)
    #plt.show()

gdf = gpd.read_file(shapefile_path)
states = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip")
states = states.to_crs(epsg=3857)
counties = gpd.read_file("https://www.weather.gov/source/gis/Shapefiles/County/c_05mr24.zip")
counties = counties.to_crs(epsg=3857)

for i in range(len(formatted_start_times)):
    start_time = formatted_start_times[i]
    end_time = formatted_end_times[i]
    grid_gdf = make_grid(gdf, start_time, end_time)
    # Check if make_grid returned None due to an empty GeoDataFrame
    if grid_gdf is not None:
        plot_grid(grid_gdf, start_time, end_time)


# Run command to create animation
cmd = f'convert -delay 20 -loop 0 *.png polygon_animation.gif'
os.system(cmd)
